In [1]:
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jacobemerson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jacobemerson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jacobemerson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jacobemerson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
jobs_df = pd.read_csv("/Users/jacobemerson/code/cgallin/cover_genie/raw_data/job_data/postings.csv")
jobs_df.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [3]:
# def preprocessor(df):
#     def text_cleaner(df):
#         # Ensure 'description' column exists
#         if 'description' not in df.columns:
#             raise ValueError("DataFrame must contain a 'description' column.")
        
#         # Drop rows with missing 'description' values
#         df_cleaned = df.dropna(subset=['description'])
        
#         # Convert text to lowercase and remove punctuation
#         translator = str.maketrans('', '', string.punctuation)
#         df_cleaned['description'] = df_cleaned['description'].str.lower().str.translate(translator)
        
#         # Remove numbers
#         df_cleaned['description'] = df_cleaned['description'].str.replace(r'\d+', '', regex=True)
        
#         return df_cleaned['description']

#     text = text_cleaner(df)

#     # Tokenize the text
#     def tokenizer(texts):
#         return texts.apply(word_tokenize)

#     tokens = tokenizer(text)

#     # Remove stopwords
#     def remove_stopwords(tokens):
#         stop_words = set(stopwords.words('english')).union(stopwords.words('french'))
#         return tokens.apply(lambda words: [w for w in words if w not in stop_words])

#     no_stopword_tokens = remove_stopwords(tokens)

#     # Lemmatize text
#     def lemmatize(tokens):
#         lemmatizer = WordNetLemmatizer()
        
#         def lemmatize_words(words):
#             # Lemmatize verbs and nouns
#             lemmatized_verbs = [lemmatizer.lemmatize(word, pos='v') for word in words]
#             lemmatized_nouns = [lemmatizer.lemmatize(word, pos='n') for word in lemmatized_verbs]
#             return lemmatized_nouns
        
#         return tokens.apply(lemmatize_words)

#     lemmatized_tokens = lemmatize(no_stopword_tokens)

#     # Return lemmatized tokens as a new DataFrame column
#     df['processed_description'] = lemmatized_tokens
#     return pd.DataFrame(df['processed_description'])
    


In [4]:
# def preprocessor(df):
#     # Clean and preprocess text
#     def text_cleaner(df):
#         if 'description' not in df.columns:
#             raise ValueError("DataFrame must contain a 'description' column.")
        
#         # Drop rows with missing descriptions
#         df_cleaned = df.dropna(subset=['description']).copy()
        
#         # Apply regex to clean text (keep only letters, replace others with space)
#         df_cleaned['description'] = df_cleaned['description'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
        
#         # Convert to lowercase
#         df_cleaned['description'] = df_cleaned['description'].str.lower()
        
#         return df_cleaned['description']

#     # Process sentences within the cleaned text
#     def process_sentences(texts):
#         # Set up stopwords and lemmatizer
#         stop_words = set(stopwords.words('english')).union(stopwords.words('french'))
#         lemmatizer = WordNetLemmatizer()
        
#         # Extract features from sentences
#         def extract_features(text):
#             features = {'feature': ""}
#             sentences = sent_tokenize(text)
#             for sent in sentences:
#                 # Tokenize, remove stopwords, and filter by POS tags
#                 words = word_tokenize(sent)
#                 words = [word for word in words if word not in stop_words]
#                 tagged_words = pos_tag(words)
#                 filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
                
#                 # Lemmatize remaining words
#                 lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in filtered_words]
#                 lemmatized_words = [lemmatizer.lemmatize(word, pos='n') for word in lemmatized_words]
                
#                 # Append to features
#                 features['feature'] += " ".join(lemmatized_words) + " "
#             return features['feature']
        
#         # Apply feature extraction to all texts
#         return texts.apply(extract_features)
    
#     # Apply text cleaning
#     cleaned_text = text_cleaner(df)
    
#     # Process sentences and extract features
#     processed_text = process_sentences(cleaned_text)
    
#     # Add processed text as a new column
#     df['processed_description'] = processed_text
#     return df[['processed_description']].copy()

In [5]:
def preprocessor(text):
    # Clean the text
    def text_cleaner(text):
        # Apply regex to clean text (keep only letters, replace others with space)
        cleaned_text = re.sub('[^a-zA-Z]', ' ', text)
        
        # Convert to lowercase
        cleaned_text = cleaned_text.lower()
        
        return cleaned_text

    # Process sentences within the cleaned text
    def process_sentences(text):
        # Set up stopwords and lemmatizer
        stop_words = set(stopwords.words('english')).union(stopwords.words('french'))
        lemmatizer = WordNetLemmatizer()
        
        # Extract features from sentences
        def extract_features(text):
            features = ""
            sentences = sent_tokenize(text)
            for sent in sentences:
                # Tokenize, remove stopwords, and filter by POS tags
                words = word_tokenize(sent)
                words = [word for word in words if word not in stop_words]
                tagged_words = pos_tag(words)
                filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
                
                # Lemmatize remaining words
                lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in filtered_words]
                lemmatized_words = [lemmatizer.lemmatize(word, pos='n') for word in lemmatized_words]
                
                # Append to features
                features += " ".join(lemmatized_words) + " "
            return features.strip()
        
        # Apply feature extraction to the text
        return extract_features(text)
    
    # Apply text cleaning
    cleaned_text = text_cleaner(text)
    
    # Process sentences and extract features
    processed_text = process_sentences(cleaned_text)
    
    return processed_text


In [6]:
# Apply the preprocessor function to the first 100 rows of the 'description' column
jobs_df.loc[:99, 'processed_description'] = jobs_df.loc[:99, 'description'].apply(preprocessor)


In [7]:
resumes_df = pd.read_csv('/Users/jacobemerson/code/cgallin/cover_genie/raw_data/archive-3/Resume/Resume.csv')

In [8]:
resumes_df = resumes_df.rename(columns={'Resume_str':'description'})

In [9]:
resumes_df.loc[:99, 'processed_description'] = resumes_df.loc[:99, 'description'].apply(preprocessor)

In [10]:
# # Combine all text data for fitting the vectorizer
# resumes = resumes_df.loc[:99, 'processed_description'].str.split().sum()
# job_desc = jobs_df.loc[:99, 'processed_description'].str.split().sum()

In [11]:
# # Vectorizing the text data
# vectorizer = TfidfVectorizer()
# job_desc_tfidf = vectorizer.fit_transform(resumes_df.loc[:99, 'processed_description'])
# resume_tfidf = vectorizer.transform(jobs_df.loc[:99, 'processed_description'])

In [12]:
# from sklearn.metrics.pairwise import cosine_similarity

# tfidf_similarity_matrix = cosine_similarity(job_desc_tfidf, resume_tfidf)

In [13]:
# # Define the number of top jobs to review
# k = 5

# for resume_idx in range(tfidf_similarity_matrix.shape[1]):  # Loop over resumes
#     top_jobs = tfidf_similarity_matrix[:, resume_idx].argsort()[::-1][:k]
#     print(f"Resume {resume_idx}: Top {k} jobs")
#     for job_idx in top_jobs:
#         similarity_score = tfidf_similarity_matrix[job_idx, resume_idx]
#         print(f"  Job {job_idx}, Similarity: {similarity_score:.4f}")

In [14]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# def recommendation(resume_text, job_text, k=5):
#     """
#     Recommends top k jobs for each resume based on text similarity.

#     Parameters:
#     - resume_text: List or Series of resume descriptions (processed).
#     - job_text: List or Series of job descriptions (processed).
#     - k: Number of top job recommendations for each resume.

#     Returns:
#     - None (prints recommendations for each resume).
#     """
#     # Ensure inputs are lists of strings
#     if isinstance(resume_text, list):
#         resume_text = pd.Series(resume_text)
#     if isinstance(job_text, list):
#         job_text = pd.Series(job_text)

#     # Vectorizing the text data
#     vectorizer = TfidfVectorizer()
#     job_desc_tfidf = vectorizer.fit_transform(job_text)
#     resume_tfidf = vectorizer.transform(resume_text)

#     # Compute the cosine similarity between job descriptions and resumes
#     tfidf_similarity_matrix = cosine_similarity(job_desc_tfidf, resume_tfidf)

#     # Iterate through each resume and find the top k most similar jobs
#     for resume_idx in range(tfidf_similarity_matrix.shape[1]):  # Loop over resumes
#         top_jobs = tfidf_similarity_matrix[:, resume_idx].argsort()[::-1][:k]
#         print(f"Resume {resume_idx}: Top {k} jobs")
#         for job_idx in top_jobs:
#             similarity_score = tfidf_similarity_matrix[job_idx, resume_idx]
#             print(f"  Job {job_idx}, Similarity: {similarity_score:.4f}")


In [15]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# def recommendation(resume_text, job_text, k=5):
#     """
#     Recommends top k jobs for a given resume based on text similarity.

#     Parameters:
#     - resume_text: A single resume description (processed).
#     - job_text: List or Series of job descriptions (processed).
#     - k: Number of top job recommendations.

#     Returns:
#     - List of tuples containing job indices and similarity scores for the top k jobs.
#     """
#     # Ensure inputs are in the correct format
#     if isinstance(job_text, list):
#         job_text = pd.Series(job_text)

#     # Vectorizing the text data
#     vectorizer = TfidfVectorizer()
#     job_desc_tfidf = vectorizer.fit_transform(job_text)
#     resume_tfidf = vectorizer.transform([resume_text])  # Transform single resume

#     # Compute the cosine similarity between the resume and all job descriptions
#     tfidf_similarity_scores = cosine_similarity(job_desc_tfidf, resume_tfidf).flatten()

#     # Find the top k most similar jobs
#     top_jobs = tfidf_similarity_scores.argsort()[::-1][:k]
#     top_job_scores = [(job_idx, tfidf_similarity_scores[job_idx]) for job_idx in top_jobs]

#     return top_job_scores


In [16]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# def recommendation(resume_text, job_text, df, k=5):
#     """
#     Recommends top k jobs for a given resume based on text similarity.

#     Parameters:
#     - resume_text: A single resume description or list of resume descriptions (processed).
#     - job_text: A list or Series of job descriptions (processed).
#     - k: Number of top job recommendations.

#     Returns:
#     - List of tuples containing job indices and similarity scores for the top k jobs.
#     """
#     # Ensure that resume_text is a list of strings
#     if isinstance(resume_text, str):  # If it's a single resume string
#         resume_text = [resume_text]  # Convert to a list

#     if isinstance(job_text, pd.Series):  # If job_text is a pandas Series
#         job_text = job_text.tolist()  # Convert to a list of strings

#     # Ensure job_text is a list of strings
#     if not isinstance(job_text, list):
#         raise ValueError("job_text should be a list of strings.")

#     # Vectorizing the text data
#     vectorizer = TfidfVectorizer()
#     job_desc_tfidf = vectorizer.fit_transform(job_text)

#     # Prepare a list to store top recommendations for each resume
#     all_top_job_scores = []

#     # Process each resume in resume_text
#     for resume in resume_text:
#         resume_tfidf = vectorizer.transform([resume])  # Transform the single resume

#         # Compute the cosine similarity between the resume and all job descriptions
#         tfidf_similarity_scores = cosine_similarity(job_desc_tfidf, resume_tfidf).flatten()

#         # Find the top k most similar jobs
#         top_jobs = tfidf_similarity_scores.argsort()[::-1][:k]
#         top_job_scores = [(job_idx, tfidf_similarity_scores[job_idx]) for job_idx in top_jobs]

#         # Append the top job scores for the current resume
#         all_top_job_scores.append(top_job_scores)
    
#     indices = [job[0] for job in all_top_job_scores]
        
#     def index_rows_by_values(df, indices):
#         """
#         Indexes the rows of a DataFrame by an array of row indices.

#         Parameters:
#         - df (pd.DataFrame): The DataFrame to index.
#         - indices (list or array-like): A list or array of row indices to select.

#         Returns:
#         - pd.DataFrame: A new DataFrame with only the selected rows.
#         """
#         if not isinstance(df, pd.DataFrame):
#             raise ValueError("Input 'df' must be a pandas DataFrame.")
        
#         if not all(isinstance(i, int) for i in indices):
#             raise ValueError("All elements in 'indices' must be integers.")
        
#         top_jobs_df = df.iloc[indices]
        
#         return top_jobs_df[['title', 'company_name', 'description']]
    
#     top_jobs_df = index_rows_by_values(df, indices)

#     return top_jobs_df


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommendation(resume_text, job_text, df, k=5):
    """
    Recommends top k jobs for a given resume based on text similarity and retrieves job details.

    Parameters:
    - resume_text: A single resume description or list of resume descriptions (processed).
    - job_text: A list or Series of job descriptions (processed).
    - df: A DataFrame containing job details with columns 'title', 'company_name', 'description'.
    - k: Number of top job recommendations.

    Returns:
    - pd.DataFrame: A DataFrame containing top job recommendations with 'title', 'company_name', 'description'.
    """
    # Ensure that resume_text is a list of strings
    if isinstance(resume_text, str):  # If it's a single resume string
        resume_text = [resume_text]  # Convert to a list

    if isinstance(job_text, pd.Series):  # If job_text is a pandas Series
        job_text = job_text.tolist()  # Convert to a list of strings

    # Ensure job_text is a list of strings
    if not isinstance(job_text, list):
        raise ValueError("job_text should be a list of strings.")

    # Vectorizing the text data
    vectorizer = TfidfVectorizer()
    job_desc_tfidf = vectorizer.fit_transform(job_text)

    # Prepare to store all DataFrames for top recommendations
    all_top_jobs_df = []

    # Process each resume in resume_text
    for resume in resume_text:
        resume_tfidf = vectorizer.transform([resume])  # Transform the single resume

        # Compute the cosine similarity between the resume and all job descriptions
        tfidf_similarity_scores = cosine_similarity(job_desc_tfidf, resume_tfidf).flatten()

        # Find the top k most similar jobs
        top_jobs = tfidf_similarity_scores.argsort()[::-1][:k]

        # Index rows from the DataFrame
        top_jobs_df = df.iloc[top_jobs][['title', 'company_name', 'description']]

        # Append to the list of results
        all_top_jobs_df.append(top_jobs_df)

    # Return the results as a list of DataFrames (one per resume) or a concatenated DataFrame
    return all_top_jobs_df if len(all_top_jobs_df) > 1 else all_top_jobs_df[0]


In [18]:
# Extract the first 100 processed descriptions
resume_text = resumes_df.loc[:99, 'processed_description']
job_text = jobs_df.loc[:99, 'processed_description']


# # Iterate through each resume and get recommendations
# for idx, resume_text in enumerate(resume_text):
#     print(f"Recommendations for Resume {idx}:")
#     top_jobs = recommendation(resume_text, job_text, k=5)
#     for job_idx, similarity in top_jobs:
#         print(f"  Job {job_idx}, Similarity: {similarity:.4f}")

In [19]:
# Extract the first 100 processed descriptions
resume_text = resumes_df.loc[0, 'processed_description']
job_text = jobs_df.loc[:99, 'processed_description']

rec = recommendation(resume_text, job_text, jobs_df, k=5)

In [20]:
rec

,title,company_name,description
99,Marketing Coordinator,Lynx Systems,Micro Technology Services Inc. is a leading pr...
35,Marketing & Office Coordinator,Revesco Properties,About Revesco Properties:Revesco Properties is...
95,Service Coordinator,Grunwald Mechanical Contractors & Engineers,"QualificationsExperience:Data Entry, 4 years (..."
14,Senior Product Marketing Manager,Staffing Theory,A leading pharmaceutical company committed to ...
30,Administrative Assistant,REquipment Durable Medical Equipment and Assis...,The Administrative Assistant will organize and...


In [21]:
indices = [job[0] for job in rec]



In [22]:
def index_rows_by_values(df, indices):
    """
    Indexes the rows of a DataFrame by an array of row indices.

    Parameters:
    - df (pd.DataFrame): The DataFrame to index.
    - indices (list or array-like): A list or array of row indices to select.

    Returns:
    - pd.DataFrame: A new DataFrame with only the selected rows.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input 'df' must be a pandas DataFrame.")
    
    if not all(isinstance(i, int) for i in indices):
        raise ValueError("All elements in 'indices' must be integers.")
    
    top_jobs_df = df.iloc[indices]
    
    return top_jobs_df[['title', 'company_name', 'description']]
    

In [23]:
# jobs_df.loc[16, ['description', 'title']]

In [24]:
# print(jobs_df['description'][16])

In [25]:
# print(resumes_df['description'][99]